In [ ]:
import py2neo
py2neo.__version__

'5.0b1'

## Create

### Step 0: Connect to Graph using Py2Neo

In [ ]:
from py2neo import Graph
graph = Graph("bolt://localhost:11003", password='1234')

### Step 1: Create Node and Relationship Subgraphs using Py2Neo

This is just a sample -- the full dataset is quite long and would be silly to keep here ([jump](#Full-Movie-Graph-Create)), it's at the end of this notebook. It'll be necessary to have all the data to run some of the examples.

[Full Create](#Full-Movie-Graph-Create)

In [ ]:
from py2neo import Node, Relationship

# Nodes
TheMatrix = Node("Movie", title='The Matrix', released=1999, tagline='Welcome to the Real World')
Keanu = Node("Person", name='Keanu Reeves', born=1964)
Carrie = Node("Person", name='Carrie-Anne Moss', born=1967)
Laurence = Node("Person", name='Laurence Fishburne', born=1961)
Hugo = Node("Person", name='Hugo Weaving', born=1960)
LillyW = Node("Person", name='Lilly Wachowski', born=1967)
LanaW = Node("Person", name='Lana Wachowski', born=1965)
JoelS = Node("Person", name='Joel Silver', born=1952)
Emil = Node("Person", name="Emil Eifrem", born=1978)

# Relationships
LillyWTheMatrix = Relationship(LillyW, "DIRECTED", TheMatrix)
LanaWTheMatrix = Relationship(LanaW, "DIRECTED", TheMatrix)
JoelSTheMatrix = Relationship(JoelS, "PRODUCED", TheMatrix)
KeanuTheMatrix = Relationship(Keanu, "ACTED_IN", TheMatrix)
KeanuTheMatrix['roles'] = ['Neo']
CarrieTheMatrix = Relationship(Carrie, "ACTED_IN", TheMatrix)
CarrieTheMatrix['roles'] = ['Trinity']
LaurenceTheMatrix = Relationship(Laurence, "ACTED_IN", TheMatrix)
LaurenceTheMatrix['roles'] = ['Morpheus']
HugoTheMatrix = Relationship(Hugo, "ACTED_IN", TheMatrix)
HugoTheMatrix['roles'] = ['Agent Smith']
EmilTheMatrix = Relationship(Emil, "ACTED_IN", TheMatrix)
EmilTheMatrix['roles'] = ['Emil']

### Step 2: Commit using Py2Neo

In [ ]:
tx = graph.begin()
tx.create(TheMatrix)
tx.create(Keanu)
tx.create(Carrie)
tx.create(Laurence)
tx.create(Hugo)
tx.create(LillyW)
tx.create(LanaW)
tx.create(JoelS)
tx.create(Emil)
tx.create(KeanuTheMatrix)
tx.create(CarrieTheMatrix)
tx.create(LaurenceTheMatrix)
tx.create(HugoTheMatrix)
tx.create(LillyWTheMatrix)
tx.create(LanaWTheMatrix)
tx.create(JoelSTheMatrix)
tx.create(EmilTheMatrix)
tx.commit()

## Find

In [ ]:
from py2neo.matching import NodeMatcher
node_matcher = NodeMatcher(graph)
keanu = node_matcher.match("Person", name="Keanu Reeves").first()
keanu

Node('Person', born=1964, name='Keanu Reeves')

In [ ]:
keanu0 = graph.nodes[1]
keanu1 = graph.nodes.get(1)
keanu2 = graph.nodes.match("Person", name="Keanu Reeves").first()
    
keanu0 == keanu1 == keanu2

True

Node('Person', born=1964, name='Keanu Reeves')

In [ ]:
match_using_matcher = node_matcher.match(name="Keanu Reeves").first()
match_using_graphnodes = graph.nodes.match(name="Keanu Reeves").first()

assert match_using_matcher == match_using_graphnodes
print(match_using_matcher == match_using_graphnodes)

True


In [ ]:
len(graph.nodes.match("Person"))

145

#### Find the actor named "Tom Hanks"...

In [ ]:
node_matcher.match(name="Tom Hanks").first()

Node('Person', born=1956, name='Tom Hanks')

In [ ]:
graph.nodes.match(name="Tom Hanks").first()

Node('Person', born=1956, name='Tom Hanks')

#### Find the movie with title "Cloud Atlas"...

In [ ]:
node_matcher.match(title="Cloud Atlas").first()

Node('Movie', released=2012, tagline='Everything is connected', title='Cloud Atlas')

In [ ]:
graph.nodes.match(title="Cloud Atlas").first()

#### Find 10 people...

In [ ]:
node_matcher.match("Person").limit(10).all()

In [ ]:
graph.nodes.match("Person").limit(10).all()

#### Find movies released in the 1990s...

In [ ]:
graph.nodes.match("Movie").where('_.released >= 1990', '_.released < 2000').all()

[Node('Movie', released=1999, tagline='Welcome to the Real World', title='The Matrix'),
 Node('Movie', released=1999, tagline='Welcome to the Real World', title='The Matrix'),
 Node('Movie', released=1992, tagline="In the heart of the nation's capital, in a courthouse of the U.S. government, one man will stop at nothing to keep his honor, and one will stop at nothing to find the truth.", title='A Few Good Men'),
 Node('Movie', released=1992, tagline='Once in a lifetime you get a chance to do something different.', title='A League of Their Own'),
 Node('Movie', released=1995, tagline='Houston, we have a problem.', title='Apollo 13'),
 Node('Movie', released=1997, tagline='A comedy from the heart that goes for the throat.', title='As Good as It Gets'),
 Node('Movie', released=1999, tagline="One robot's 200 year journey to become an ordinary man.", title='Bicentennial Man'),
 Node('Movie', released=1992, tagline="He didn't want law. He wanted justice.", title='Hoffa'),
 Node('Movie', rele

#### Full Movie Graph Create